In [65]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from html_table_parser import parser_functions


def std_crawler(url):
    driver = webdriver.Chrome(executable_path=r"C:\Users\gihok\chatbot\chromedriver.exe")
    driver.get(url)
    time.sleep(3)
    
    df = pd.DataFrame(columns=['표준번호','표준명(한글)','대체 표준','표준명(영문)','KS원문보기','표준분야','표준구분','제정일','최종개정확인일',
                               '기술심의회','전문위원회','적용범위','표준개발협력기관','담당부처','담당부서','담당자명',
                               '담당자 연락처','국제표준 부합화','인용표준','기술기준','ICS Code','표준이력사항','인증심사기준'])
    while True:
        try:
            # 페이지 넘기기
            # for i in range(3, 14):
            for i in range(3,6):
                driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[3]/div/div/ul/li[{}]/a'.format(i)).click()
                
                time.sleep(3)
                
                # 표준 들어가기
                for j in range(1, 11):
                    driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[2]/table/tbody/tr[{}]/td[2]/a'.format(j)).click()
                    
                    time.sleep(3)
                    
                    #crawling 작업
                    try:
                        dict = {}
                        for z in range(1, 22):
                            # 인용표준 및 기술기준
                            if z == 18:
                                title_1 = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[{}]/th[1]'.format(z)).text
                                content_1 = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[18]/td[1]').text
                                title_2 = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[{}]/th[2]'.format(z)).text
                                content_2 = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[18]/td[2]').text
                                dict[title_1] = [content_1]
                                dict[title_2] = [content_2]
                            
                            #테이블 데이터 처리
                            elif z == 17 or z == 20 or z == 21:
                                title = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[{}]/th[1]'.format(z)).text
                                result_html = driver.page_source
                                soup = BeautifulSoup(result_html, 'html.parser')
                                # 국제표준 부합화
                                if z == 17:
                                    data = soup.find("div", {'class':"table list gray"})
                                #표준이력사항
                                elif z == 20:
                                    data = soup.find_all("div", {'class':"table list gray"})[1]
                                #인증심사기준
                                else:
                                    data = soup.find_all("div", {'class':"table list gray"})[2]
                                    
                                table = parser_functions.make2d(data)
                                df_std = pd.DataFrame(data = table[1:], columns = table[0])
                                dict[title] = [df_std]
                                
                            else:    
                                title = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[{}]/th[1]'.format(z)).text
                                content = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[{}]/td[1]'.format(z)).text
                                dict[title] = [content]
                                
                        dict_df = pd.DataFrame(dict)
                        df = pd.concat([df, dict_df], ignore_index=True)
                    
                    except:
                        break
                    
                    driver.back()
                    driver.refresh()
                    time.sleep(3)
                    
            return df    
        except:
            break
        
    return df

In [66]:
df = std_crawler("https://standard.go.kr/KSCI/standardIntro/getStandardSearchList.do?menuId=919&topMenuId=502")

C:\Users\gihok\AppData\Local\Temp\ipykernel_117972\1397614501.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\gihok\chatbot\chromedriver.exe")


In [67]:
df

,표준번호,표준명(한글),대체 표준,표준명(영문),KS원문보기,표준분야,표준구분,제정일,최종개정확인일,기술심의회,...,담당부처,담당부서,담당자명,담당자 연락처,국제표준 부합화,인용표준,기술기준,ICS Code,표준이력사항,인증심사기준
0,KS A 0001,표준서의 서식 및 작성방법,NaN,Rules for the drafting and presentation of Kor...,표준 원문보기,(A) 기본 - (A10) 기본일반,방법,1962-06-26,2021-01-11,기계기본 기술심의회(B),...,국가기술표준원,산업표준혁신과,고정하,--,대응국제표준 부합화수준 0 해당없음 해당없음,147 건,1 건,01.120 (표준화. 일반 규칙),변경일자 구분 고시번호 \ 0 1962-06-2...,제/개정일자 구분 첨부파일 0 ...
1,KS A 0002 (폐지),법정계량 용어 (폐지),,International vocabulary of legal metrology,NaN,(A) 기본 - (A17) 기타,전달,2004-12-24,2019-12-31,기계기본 기술심의회(B),...,국가기술표준원,기계융합산업표준과,양승배,043-870-5373,대응국제표준 부합화수준 0 해당없음 해당없음,0 건,2 건,해당없음,변경일자 구분 고시번호 \ 0 2004-12-24 ...,제/개정일자 구분 첨부파일 0 ...
2,KS A 0003 (폐지),법정계량 용어 (폐지),KSA0002,Vocabulary of terms in legal metrology,NaN,(A) 기본 - (A10) 기본일반,전달,2004-12-24,2013-11-05,기계기본 기술심의회(B),...,국가기술표준원,기계융합산업표준과,양승배,043-870-5373,대응국제표준 부합화수준 0 관련 자료가 없습니다. ...,0 건,0 건,01.040.17 (계랑 및 측정. 물리현상 (용어)),변경일자 구분 고시번호 \ 0 2004-12-24 ...,제/개정일자 구분 첨부파일 0 ...
3,KS A 0004 (폐지),분석 . 시험의 허용차 통칙 (폐지),,GENERAL RULES FOR PERMISSIBLE TOLERANCE OF CHE...,NaN,(A) 기본 -,전달,1981-12-11,2002-12-30,,...,국가기술표준원,기계융합산업표준과,이경희,043-870-5379,대응국제표준 부합화수준 0 ISO 5725-1 NEQ ...,0 건,8 건,,"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표...",제/개정일자 구분 첨부파일 0 ...
4,KS A 0005,제도 ― 통칙(도면 작성의 일반코드),NaN,TECHNICAL DRAWING-GENERAL CODE OF DRAWING PRAC...,표준 원문보기,(A) 기본 - (A10) 기본일반,전달,1969-10-25,2019-12-31,기계기본 기술심의회(B),...,국가기술표준원,기계융합산업표준과,김민우,043-870-5376,대응국제표준 부합화수준 0 해당없음 해당없음,87 건,1 건,01.100.00 (기술도안 일반),변경일자 구분 고시번호 \ 0 1969-10-2...,제/개정일자 구분 첨부파일 0 ...
5,KS A 0006,시험 장소의 표준 상태,NaN,STANDARD ATMOSPHERIC CONDITIONS FOR TESTING,표준 원문보기,(A) 기본 - (A10) 기본일반,전달,1971-05-22,2019-12-24,기계기본 기술심의회(B),...,국가기술표준원,기계융합산업표준과,김민우,043-870-5376,대응국제표준 부합화수준 0 ISO 554:1976 IDT 일치,0 건,64 건,19.020 (시험 조건 및 절차 일반),변경일자 구분 고시번호 ...,제/개정일자 구분 첨부파일 0 ...
6,KS A 0007 (폐지),음향 용어 ( 기기 ) (폐지),,GLOSSARY OF ACOUSTICAL TERMS (TRANSDUCERS AND ...,NaN,(A) 기본 -,전달,1982-11-04,2001-11-19,전기응용 기술심의회(C),...,국가기술표준원,전기전자정보표준과,김성희,043-870-5365,대응국제표준 부합화수준 0 관련 자료가 없습니다. ...,0 건,0 건,01.040.17 (계랑 및 측정. 물리현상 (용어)) 17.140.00 (음향 및...,"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표...",제/개정일자 구분 첨부파일 0 ...
7,KS A 0009 (폐지),음향 용어 ( 음성 . 청각 . 음악 ) (폐지),,GLOSSARY OF ACOUSTICAL TERMS (HEARING.SPEECH.M...,NaN,(A) 기본 -,전달,1982-11-04,2001-11-19,전기응용 기술심의회(C),...,국가기술표준원,전기전자정보표준과,김성희,043-870-5365,대응국제표준 부합화수준 0 관련 자료가 없습니다. ...,0 건,0 건,01.040.17 (계랑 및 측정. 물리현상 (용어)) 17.140.01 (음향 측...,"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표...",제/개정일자 구분 첨부파일 0 ...
8,KS A 0010,오염 물질 조정 용어,NaN,Glossary of terms for contamination control,표준 원문보기,(A) 기본 - (A10) 기본일반,전달,1985-05-08,2020-12-30,산업환경 기술심의회(I),...,국가기술표준원,바이오화학서비스표준과,지민호,043-870-5394,대응국제표준 부합화수준 0 해당없음 해당없음,2 건,0 건,01.040.13 (환경 및 보건. 안전도 (용어)),"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표...",제/개정일자 구분 첨부파일 0 ...
9,KS A 0011,물체색의 색 이름,NaN,Names of non - luminous object colours,표준 원문보기,(A) 기본 - (A10) 기본일반,전달,1967-12-30,2020-12-22,문화 기술심의회(A),...,국가기술표준원,바이오화학서비스표준과,박정은,043-870-5401,대응국제표준 부합화수준 0 해당없음 해당없음,6 건,8 건,17.180.20 (빛의 색 및 측정),"변경일자 구분 고시번호 제정,개정,...",제/개정일자 구분 첨부파일 0 ...


In [68]:
df.loc[3]

표준번호                                           KS A 0004 (폐지)
표준명(한글)                                  분석 . 시험의 허용차 통칙 (폐지)
대체 표준                                                        
표준명(영문)     GENERAL RULES FOR PERMISSIBLE TOLERANCE OF CHE...
KS원문보기                                                    NaN
표준분야                                                 (A) 기본 -
표준구분                                                       전달
제정일                                                1981-12-11
최종개정확인일                                            2002-12-30
기술심의회                                                        
전문위원회                                                        
적용범위                                                         
표준개발협력기관                                                     
담당부처                                                  국가기술표준원
담당부서                                                기계융합산업표준과
담당자명                                                      이경희
담당자 연락처 

In [70]:
df.loc[3]['국제표준 부합화']

,대응국제표준,부합화수준
0,ISO 5725-1,NEQ 동일하지 않음
1,ISO 5725-2,NEQ 동일하지 않음
2,ISO 5725-3,NEQ 동일하지 않음
3,ISO 5725-4,NEQ 동일하지 않음
4,ISO 5725-6,NEQ 동일하지 않음


In [71]:
df.loc[3]['표준이력사항']

,변경일자,구분,고시번호,"제정,개정,폐지 사유",신구대비표
0,1981-12-11,개정,811985,,
1,1986-10-27,확인,861819,,
2,1991-03-06,확인,910097,,
3,1996-12-27,확인,960194,,
4,1997-06-19,개정,97122,,
5,2002-12-30,폐지,2002-1794,,


In [72]:
df.loc[3]['인증심사기준']

,제/개정일자,구분,첨부파일
0,관련 자료가 없습니다.,관련 자료가 없습니다.,관련 자료가 없습니다.
